In [25]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms.v2 as transforms
import torchvision.io as tv_io

import glob
from PIL import Image

import utils

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

True

In [26]:
from torchvision.models import vgg16
from torchvision.models import VGG16_Weights

weights = VGG16_Weights.DEFAULT
vgg_model = vgg16(weights=weights)

In [46]:
# Freeze base model
vgg_model.requires_grad_(True)
next(iter(vgg_model.parameters())).requires_grad

True

In [47]:
N_CLASSES = 6

my_model = nn.Sequential(
    vgg_model.features,
    vgg_model.avgpool,
    nn.Flatten(),
    vgg_model.classifier[0:3],
    nn.Linear(4096, 500),
    nn.ReLU(),
    nn.Linear(500, N_CLASSES)
)
my_model

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [48]:
loss_function = nn.CrossEntropyLoss()
optimizer = Adam(my_model.parameters(), lr=.0001)
my_model = torch.compile(my_model.to(device))

In [49]:
pre_trans = weights.transforms()

In [56]:
IMG_WIDTH, IMG_HEIGHT = (224, 224)

random_trans = transforms.Compose([
    transforms.RandomRotation(25),
    transforms.RandomResizedCrop((IMG_WIDTH, IMG_HEIGHT), scale=(0.8, 1), ratio=(0.75, 1.33)),  # Variasi rasio
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.3, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomVerticalFlip(),  # Menambahkan flip vertikal
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalisasi
])

In [57]:
DATA_LABELS = ["freshapples", "freshbanana", "freshoranges", "rottenapples", "rottenbanana", "rottenoranges"] 
    
class MyDataset(Dataset):
    def __init__(self, data_dir):
        self.imgs = []
        self.labels = []
        
        for l_idx, label in enumerate(DATA_LABELS):
            data_paths = glob.glob(data_dir + label + '/*.png', recursive=True)
            for path in data_paths:
                img = tv_io.read_image(path, tv_io.ImageReadMode.RGB)
                self.imgs.append(pre_trans(img).to(device))
                self.labels.append(torch.tensor(l_idx).to(device))


    def __getitem__(self, idx):
        img = self.imgs[idx]
        label = self.labels[idx]
        return img, label

    def __len__(self):
        return len(self.imgs)

In [58]:
n = 64

train_path = "data/fruits/train/"
train_data = MyDataset(train_path)
train_loader = DataLoader(train_data, batch_size=n, shuffle=True)
train_N = len(train_loader.dataset)

valid_path = "data/fruits/valid/"
valid_data = MyDataset(valid_path)
valid_loader = DataLoader(valid_data, batch_size=n, shuffle=False)
valid_N = len(valid_loader.dataset)

In [59]:
epochs = 20

for epoch in range(epochs):
    print('Epoch: {}'.format(epoch))
    utils.train(my_model, train_loader, train_N, random_trans, optimizer, loss_function)
    utils.validate(my_model, valid_loader, valid_N, loss_function)

Epoch: 0
Train - Loss: 1.2485 Accuracy: 0.9805
Valid - Loss: 3.7365 Accuracy: 0.8237
Epoch: 1
Train - Loss: 0.5247 Accuracy: 0.9898
Valid - Loss: 2.9502 Accuracy: 0.8511
Epoch: 2
Train - Loss: 0.6375 Accuracy: 0.9856
Valid - Loss: 1.8463 Accuracy: 0.9179
Epoch: 3
Train - Loss: 0.7592 Accuracy: 0.9848
Valid - Loss: 1.6497 Accuracy: 0.9210
Epoch: 4
Train - Loss: 1.7339 Accuracy: 0.9788
Valid - Loss: 3.3098 Accuracy: 0.8754
Epoch: 5
Train - Loss: 0.9207 Accuracy: 0.9856
Valid - Loss: 4.2227 Accuracy: 0.8389
Epoch: 6
Train - Loss: 0.3532 Accuracy: 0.9932
Valid - Loss: 3.0852 Accuracy: 0.8845
Epoch: 7
Train - Loss: 0.5385 Accuracy: 0.9915
Valid - Loss: 2.0675 Accuracy: 0.9271
Epoch: 8
Train - Loss: 0.3358 Accuracy: 0.9949
Valid - Loss: 2.8008 Accuracy: 0.9210
Epoch: 9
Train - Loss: 0.1412 Accuracy: 0.9975
Valid - Loss: 2.1368 Accuracy: 0.9240
Epoch: 10
Train - Loss: 0.4832 Accuracy: 0.9941
Valid - Loss: 3.6353 Accuracy: 0.8967
Epoch: 11
Train - Loss: 0.4407 Accuracy: 0.9907
Valid - Loss: 2.

In [60]:
from run_assessment import run_assessment
run_assessment(my_model)

Evaluating model to obtain average accuracy...

Accuracy: 0.9362

Accuracy required to pass the assessment is 0.92 or greater.
Your average accuracy is 0.9362.

Congratulations! You passed the assessment!
See instructions below to generate a certificate.
